In [1]:
import pandas as pd
import numpy as np
import re

## ✂️ extract loai-bds and quan


### extract data from link

In [2]:
df = pd.read_csv('raw_SMALL.csv')

pattern = r'(?<=\.vn)/([^/]+/[^/]+/)'
df['data'] = df['link'].apply(lambda x: re.search(pattern, x).group(1) if re.search(pattern, x) else '')
print(df.head())


                                                link           price   area  \
0  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 449 triệu  82 m2   
1  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 499 triệu  82 m2   
2  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 500 triệu  82 m2   
3  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 500 triệu  83 m2   
4  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 500 triệu  82 m2   

   room toilet                                            address       lat  \
0  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
1  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
2  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
3  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
4  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   

        long                                    da

### split quan and loai-bds from data

In [3]:
# Split the "data" column into two columns based on the slash separator
df[['quan', 'loai_bds']] = df['data'].str.split('/', n=1, expand=True)

# Drop the original "data" column
df.drop('data', axis=1, inplace=True)
print(df.head())

                                                link           price   area  \
0  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 449 triệu  82 m2   
1  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 499 triệu  82 m2   
2  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 500 triệu  82 m2   
3  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 500 triệu  83 m2   
4  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...  3 tỷ 500 triệu  82 m2   

   room toilet                                            address       lat  \
0  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
1  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
2  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
3  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
4  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   

        long               quan              loai_

## 🧹 ['price'] - ['room'] -['toilet']

In [4]:
def convert_to_dong(string):
    if not isinstance(string, str):
        return 0
    pattern = r'(\d+)\s*(tỷ)?\s*(\d+)?\s*(triệu)?'
    string = string.replace('.', '')
    match = re.search(pattern, string)
    
    if match.group(2) and match.group(4):
        return int(match.group(1)) * 1000000000 + int(match.group(3)) * 1000000
    elif match.group(2):
        return int(match.group(1)) * 1000000000
    elif match.group(4):
        return int(match.group(1)) * 1000000
    else:
        return int(match.group(1))

def convert_to_number(string):
    numbers = re.findall(r'\d+', string)
    return numbers[0]

### 👀 Check missing values and test function convert

In [5]:
print(df.isna().sum())
print(convert_to_dong('5 tỷ'))
print(convert_to_number('210 m2'))

link        0
price       0
area        0
room        0
toilet      0
address     0
lat         0
long        0
quan        0
loai_bds    0
dtype: int64
5000000000
210


In [6]:

df['price'].fillna(0)
df['price-VND'] = df['price'].apply(convert_to_dong)
df = df.sort_values('price-VND', ascending=True)
df['room']=df['room'].apply(convert_to_number).astype('Int64')
df['toilet']=df['toilet'].apply(convert_to_number).astype('Int64')
df['area']=df['area'].apply(convert_to_number).astype('Int64')


In [7]:
df

,link,price,area,room,toilet,address,lat,long,quan,loai_bds,price-VND
407,https://mogi.vn/quan-hai-chau/mua-nha-biet-thu...,100.000 đ,120,4,6,"Hoá Sơn 10, Phường Hòa Cường Nam, Quận Hải Châ...",16.03346,108.23031,quan-hai-chau,mua-nha-biet-thu-lien-ke/,100000
406,https://mogi.vn/quan-hai-chau/mua-nha-mat-tien...,100.000 đ,120,4,5,"Bạch Đằng, Phường Hải Châu I, Quận Hải Châu, Đ...",16.07079,108.22511,quan-hai-chau,mua-nha-mat-tien-pho/,100000
288,https://mogi.vn/quan-son-tra/mua-nha-mat-tien-...,687.000 đ,100,4,4,"Sơn Trà Điện Ngọc, Nại Hiên Đông, Quận Sơn Trà...",16.09227,108.23095,quan-son-tra,mua-nha-mat-tien-pho/,687000
586,https://mogi.vn/quan-ngu-hanh-son/mua-nha-mat-...,1 triệu,95,3,4,"Đoàn Khuê, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...",16.02611,108.24381,quan-ngu-hanh-son,mua-nha-mat-tien-pho/,1000000
411,https://mogi.vn/quan-ngu-hanh-son/mua-nha-biet...,1 triệu,800,4,6,"Võ Nguyên Giáp, Phường Hòa Hải, Quận Ngũ Hành ...",16.02983,108.25417,quan-ngu-hanh-son,mua-nha-biet-thu-lien-ke/,1000000
...,...,...,...,...,...,...,...,...,...,...,...
684,https://mogi.vn/quan-ngu-hanh-son/mua-nha-mat-...,155 tỷ,300,78,79,"Đỗ Bá, Phường Mỹ An, Quận Ngũ Hành Sơn, Đà Nẵng",16.05085,108.24319,quan-ngu-hanh-son,mua-nha-mat-tien-pho/,155000000000
669,https://mogi.vn/quan-ngu-hanh-son/mua-dat-tho-...,265 tỷ,917,0,0,"Trần Bạch Đằng, Phường Mỹ An, Quận Ngũ Hành Sơ...",16.05224,108.24686,quan-ngu-hanh-son,mua-dat-tho-cu/,265000000000
1008,https://mogi.vn/quan-son-tra/mua-nha-mat-tien-...,450 tỷ,600,100,100,"Võ Nguyên Giáp, Phường Phước Mỹ, Quận Sơn Trà,...",16.05655,108.24693,quan-son-tra,mua-nha-mat-tien-pho/,450000000000
967,https://mogi.vn/quan-son-tra/mua-nha-mat-tien-...,450 tỷ,600,100,100,"Võ Nguyên Giáp, Phường Phước Mỹ, Quận Sơn Trà,...",16.05655,108.24693,quan-son-tra,mua-nha-mat-tien-pho/,450000000000


## ✂️ Drop the unnecessary columns


In [8]:
df.drop(['link','price','address'],inplace=True, axis=1)
df

,area,room,toilet,lat,long,quan,loai_bds,price-VND
407,120,4,6,16.03346,108.23031,quan-hai-chau,mua-nha-biet-thu-lien-ke/,100000
406,120,4,5,16.07079,108.22511,quan-hai-chau,mua-nha-mat-tien-pho/,100000
288,100,4,4,16.09227,108.23095,quan-son-tra,mua-nha-mat-tien-pho/,687000
586,95,3,4,16.02611,108.24381,quan-ngu-hanh-son,mua-nha-mat-tien-pho/,1000000
411,800,4,6,16.02983,108.25417,quan-ngu-hanh-son,mua-nha-biet-thu-lien-ke/,1000000
...,...,...,...,...,...,...,...,...
684,300,78,79,16.05085,108.24319,quan-ngu-hanh-son,mua-nha-mat-tien-pho/,155000000000
669,917,0,0,16.05224,108.24686,quan-ngu-hanh-son,mua-dat-tho-cu/,265000000000
1008,600,100,100,16.05655,108.24693,quan-son-tra,mua-nha-mat-tien-pho/,450000000000
967,600,100,100,16.05655,108.24693,quan-son-tra,mua-nha-mat-tien-pho/,450000000000


## 📝 Export data to csv

In [9]:
df.to_csv('clean_SMALL.csv',index=False)